爬虫
1. 找url
1.1 浏览器地址栏
1.2 找json

2. 找元素
2.1 html DOM, CSS Selector, XPath
2.2 json

3. 保存（持久化）
3.1 CSV, EXCEL, JSON
3.2 DATABASE

html div中找到id和class 
id是唯一的
class可以把同一类都找出来

反反爬
1. 登录 比较难的
2. User agent  做一个user agent池
3. 限制访问频率 随机等待；使用代理（代理池）


In [3]:
import pandas as pd
grads = pd.read_excel('作业.xlsx',index_col='id') #索引不能有重复，唯一的
grads.loc[[3120301231],['line_01','line_02','line_03','line_04','line_05']]

,line_01,line_02,line_03,line_04,line_05
id,,,,,
3120301231,327.0,208,206.0,212.0,209.0


In [4]:
grads.loc[[3120101260],['line_01','line_02','line_03','line_04','line_05']]

,line_01,line_02,line_03,line_04,line_05
id,,,,,
3120101260,411.0,264,213.0,209.0,217.0


In [5]:
from datetime import datetime
dt = datetime(2020,12,30,21,20)
print(dt)
t = dt.timestamp()
print(t)
print(datetime.fromtimestamp(t))
cday = datetime.strptime('2020-12-30', '%Y-%m-%d')
print(cday)
now = datetime.now()
print(now.strftime('%a, %b %d %H:%M'))

2020-12-30 21:20:00
1609334400.0
2020-12-30 21:20:00
2020-12-30 00:00:00
Wed, Dec 30 22:52


In [6]:
fid = open('sea2020.csv', 'r')
fid.read()
fid.close()

In [7]:
import struct
f = open('data20201.dat', 'rb')
a = f.read()
f.close()
# print(a)

In [8]:
import struct
import time
import numpy as np

In [9]:
typename = {0x102:'非压缩格式',0x202:'压缩格式'}

In [10]:
def GetA(i,j):
    c = p + q - i
    if c < 0 or c >= p + q + 1:
        d = 0
    else:
        d = A[i,c]
    return d

def SetA(i,j,val):
    c = p + j - i
    if c >= 0 and c < p + q + 1:
        A[i,c] = val

In [11]:
def Gauss1(A,B,n):
    global p,q
    print('正在消去：')
    for i in range(n):
        if A[i,i] == 0:
            print('Error!')
            return
        LastRow = i+p+1
        if LastRow >= n:
            LastRow = n
        for j in range(i+1,LastRow):
            I = A[j,i]/A[i,j]
            LastCol = i + q + 1
            if LastCol >= n:
                LastCol = n
            for k in range(i+1,LastCol):
                A[j,k] = A[j,k] - A[j,k]*I
            B[j] = B[j] - B[i] *I
    print('正在回代求解：')
    res = np.zeros([n])
    res[n-1] = B[n-1]/A[n-1,n-1]
    for i in range(n-2,-1,-1):
        S = B[i]
        for j in range(i+1,n):
            S = S - A[i,j] * res[j]
        res[i] = S / A[i,i]
    return res

In [12]:
def Gauss2():
    global A,B,n,p,q
    print('正在消去:')
    for i in range(n):
        if GetA(i,i) == 0:
            print('Error!')
            return
        LastRow = i + p +1
        if LastRow >= n:
            LastRow = n
        for j in range(i+1,LastRow):
            I = GetA(j,i)/GetA(i,i)
            LastCol = i + q + 1
            if LastCol >= n:
                LastCol = n
            for k in range(i+1,LastCol):
                ajk = GetA(j,k) - GetA(i,k) * I
                SetA(j,k,ajk)
            B[j] = B[j] - B[i] * I

    print('正在回代求解：')
    res = np.zeros([n])
    res[n-1] = B[n-1]/A[n-1,n-1]
    for i in range(n-2,-1,-1):
        S = B[i]
        for j in range(i+1,n):
            S = S - GetA(i,j) * res[j]
        res[i] = S / GetA(i,i)
    return res

In [13]:
#非压缩格式求解方程
def SolveFunction1(f,n,p,q):
    print('非压缩格式求解方程')

    A = np.zeros([n,n])
    B = np.zeros([n])

    print('读入所有的系数矩阵')
    for i in range(n):
        for j in range(n):
            A[i,j], = struct.unpack('f',f.read(4))

    print('读入右端常量')
    for i in range(n):
        B[i], = struct.unpack('f',f.read(4))

    # 利用带状矩阵特征的高斯消去求解
    x = Gauss1(A, B, n)
    if n > 30:
        nn = 30
    else:
        nn = n
    for i in range(nn):
        print('x[{}]={}'.format(i, x[i]))

In [14]:
#压缩格式求解方程
def SolveFunction2(f):
    global A,B,n,p,q
    print('压缩格式求解方程')
    A = np.zeros([n,p+q+1])
    B = np.zeros([n])
    print('读入所有的系数矩阵')
    for i in range(n):
        for j in range(p+q+1):
            A[i,j], = struct.unpack('f',f.read(4))

    print('读入右端常量')
    for i in range(n):
        B[i], = struct.unpack('f',f.read(4))

#压缩格式高斯消去法求解
    x = Gauss2()
    if n > 30:
        nn = 30
    else:
        nn = n
    for i in range(nn):
        print('x[{}]={}'.format(i, x[i]))

In [15]:
def ReadDataFile(filename):
    global p,q,n
    FName = filename
    print('\n\n数据文件：',filename)
    f = open(FName, 'rb')
    # 读入文件头部标志
    id, ver, id1 = struct.unpack('<3l', f.read(12))
    # 读入系数矩阵结构信息
    n, q, p = struct.unpack('<3l', f.read(12))
    print('id:', hex(id),'ver:',hex(ver),'id1',hex(id1))
    print('阶数：', n, '上带宽：', q, '下带宽：', p)

    StartTime = time.time()
    print('开始时间：' + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(StartTime)))

    if ver == 0x102:
        SolveFunction1(f, n, p, q)
    else:
        SolveFunction2(f)

    EndTime = time.time()

    print('开始时间：' + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(StartTime)))
    print('结束时间：' + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(EndTime)))
    print('用时：{}ms'.format(int(EndTime - StartTime) * 1000))

    f.close()

In [16]:
FName = 'data20201.dat'
ReadDataFile(FName)



数据文件： data20201.dat
id: 0xc0a8708 ver: 0x102 id1 0x0
阶数： 20 上带宽： 3 下带宽： 3
开始时间：2020-12-30 22:52:19
非压缩格式求解方程
读入所有的系数矩阵
读入右端常量
正在消去：
正在回代求解：
x[0]=-5074107.720841798
x[1]=-5167776.970311286
x[2]=-346866.39940590144
x[3]=25616424.376778148
x[4]=-432351.68824877177
x[5]=-35952190.86741691
x[6]=280642.0962346067
x[7]=51499.82858255177
x[8]=-771170.6418803866
x[9]=-15201.028390732285
x[10]=-18155.510500591892
x[11]=28398.743649032607
x[12]=15486.501575555543
x[13]=2053.7996269482564
x[14]=-1246.9606032090398
x[15]=-4974.348864572035
x[16]=-82.07204958070425
x[17]=-9.9293506296042
x[18]=-128.5240720256333
x[19]=145.13714004627613
开始时间：2020-12-30 22:52:19
结束时间：2020-12-30 22:52:19
用时：0ms


In [17]:
FName = 'data20203.dat'
ReadDataFile(FName)



数据文件： data20203.dat
id: 0xc0a8708 ver: 0x102 id1 0x0
阶数： 2048 上带宽： 4 下带宽： 4
开始时间：2020-12-30 22:52:19
非压缩格式求解方程
读入所有的系数矩阵
读入右端常量
正在消去：
正在回代求解：


E:\python\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in double_scalars
E:\python\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


x[0]=nan
x[1]=nan
x[2]=nan
x[3]=nan
x[4]=nan
x[5]=nan
x[6]=nan
x[7]=nan
x[8]=nan
x[9]=nan
x[10]=nan
x[11]=nan
x[12]=nan
x[13]=nan
x[14]=nan
x[15]=nan
x[16]=nan
x[17]=nan
x[18]=nan
x[19]=nan
x[20]=nan
x[21]=nan
x[22]=nan
x[23]=nan
x[24]=nan
x[25]=nan
x[26]=nan
x[27]=nan
x[28]=nan
x[29]=nan
开始时间：2020-12-30 22:52:19
结束时间：2020-12-30 22:52:21
用时：2000ms


In [18]:
from turtle import *
width(4)
forward(200)
right(90)
pencolor('red')
forward(100)
right(90)
pencolor('green')
forward(200)
right(90)
pencolor('green')
forward(200)
right(90)
done()

In [20]:
def drawStar(x, y):
    pu()
    goto(x, y)
    pd()
    # set heading: 0
    seth(0)
    for i in range(5):
        fd(40)
        rt(144)

for x in range(0, 250, 50):
    drawStar(x, 0)
done()

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
 
html = urlopen('https://blog.csdn.net/zzc15806/') #获取网页
bs = BeautifulSoup(html, 'html.parser') #解析网页
hyperlink = bs.find_all('a')  #获取所有超链接
for h in hyperlink:
    hh = h.get('href')
    print(hh)

javascript:;
javascript:;
javascript:;
javascript:;
javascript:;
javascript:;
javascript:;
https://blog.csdn.net/zzc15806
https://blog.csdn.net//pheapi/skin/useThemSkin?assign_skin_id=8
javascript:void(0);
https://blog.csdn.net/zzc15806/
https://blog.csdn.net/zzc15806/rss/list
https://blog.csdn.net/zzc15806/article/details/105443270
https://blog.csdn.net/zzc15806/article/details/105443270
https://blog.csdn.net/zzc15806/article/details/90510292
https://blog.csdn.net/zzc15806/article/details/90510292
https://blog.csdn.net/zzc15806/article/details/90376023
https://blog.csdn.net/zzc15806/article/details/90376023
https://blog.csdn.net/zzc15806/article/details/89920539
https://blog.csdn.net/zzc15806/article/details/89920539
https://blog.csdn.net/zzc15806/article/details/90140937
https://blog.csdn.net/zzc15806/article/details/90140937
https://blog.csdn.net/zzc15806/article/details/89135069
https://blog.csdn.net/zzc15806/article/details/89135069
https://blog.csdn.net/zzc15806/article/details/8